# Given a decision Var fix the value of one elements in one set of these decision var

- For example, given a decision var X with set across the time (t0, t1, t2, etc), the idea is set the values of the first elements of the set because in time t0 it represent the actual period

## Root folder and read env variables

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-2]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

## 2. Create gurobi model

In [ ]:
# create model
m = gp.Model('modelo')

### 3. Create decision variables
- Decision variables that are features
- Decicion variable that is the output

In [ ]:
# define set
list_set_time = ['t0', 't1', 't2', 't3', 't4', 't5', 't6']
index_set_time = pd.Index(list_set_time)
index_set_time

In [ ]:
# create decision variables - features ml model
var_X3 = gppd.add_vars(m, index_set_time, name = "decision variable Z1",
                       #lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

var_Z4 = gppd.add_vars(m, index_set_time, name = "decision variable X2",
                       #lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

In [ ]:
# crete decision variables - output ml model
var_Y2 = gppd.add_vars(m, index_set_time, name = "decision variable Y2",
                       lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

In [ ]:
# "compile"
m.update()

In [ ]:
# see decision var created
var_Y2

### 4. Define Some Constraints (extra for the objective of this example)
In this example, the following formula is defined as constraint

---------------
alpha_feature_1 = 1/5

alpha_feature_2 = 15

Y2 = alpha_feature_1 * X3 + alpha_feature_2 * 06

In [ ]:
# define parameters of the constraint
alpha_feature_x3 = 1/5
alpha_feature_z4 = 15

In [ ]:
# define function as constraint
gppd.add_constrs(m, (alpha_feature_x3 * var_X3 + alpha_feature_z4 * var_Z4), gp.GRB.EQUAL, var_Y2, name = 'function as constraint')

### 5. Define fixed values of the decision Var
- Fix the values of period t=0 for each decision var.

- t=0 represent the actual period or initial period and it in some problems and modelations is kwown

In [ ]:
# set values in t = 0

# var_X3['t0'] = 10
# gppd.add_constrs(m, var_X3[0], gp.GRB.EQUAL, 10, name = 'Initial Value X3') doesn't work :/
m.addConstr(var_X3['t0'] == 10,  name = 'Initial Value X3')

# var_Z4['t0'] = 55
m.addConstr(var_Z4['t0'] == 55,  name = 'Initial Value Z4')

In [ ]:
m

In [ ]:
m.update()

In [ ]:
m

In [ ]:
# print show one decision var
var_X3

### 6. Define objective optimization
Objetive that no generate infeasibility

In [ ]:
m.setObjective(var_X3.sum() + var_Z4.sum(),
               gp.GRB.MINIMIZE)

#### 6. Optimize and get optimal values

In [ ]:
# solve
m.optimize()

In [ ]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

In [ ]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_set_time)

######################## save optimal values - features of models (only the features) ########################

# model
solution["var_X3"] = var_X3.gppd.X
solution["var_Z4"] = var_Z4.gppd.X


######################## save optimal values - targets of models (some targets are features of the model of the next step) ########################
solution["var_Y2"] = var_Y2.gppd.X  # model


######################## # get value objetive function ########################
opt_objetive_function = m.ObjVal

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
# show value decision variables
solution

In [ ]:
#### wonderfull